内存不够，只能 train / test 分开跑了

In [1]:
import warnings
warnings.simplefilter('ignore')

import gc

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 200)
pd.set_option('float_format', lambda x: '%.6f' % x)

from tqdm import tqdm
tqdm.pandas()

In [2]:
# train = pd.read_csv('./jet_complex_data/complex_train_R04_particle.csv')
# test = pd.read_csv('./jet_complex_data/complex_test_R04_particle.csv')

# df = pd.concat([train, test], axis=0)

df = pd.read_csv('./jet_complex_data/complex_train_R04_particle.csv')

In [3]:
# train_len = len(train)

# del train, test
# gc.collect()

In [4]:
df.head()

,particle_category,particle_px,particle_py,particle_pz,particle_energy,particle_mass,jet_id
0,-211,-1.299730,0.368729,-0.057008,1.359400,0.139570,01291194f90c44c7bd79d9dbd50abd93
1,-2212,1.305420,0.418302,-0.426083,1.714930,0.938270,7df78235aa464796bbb386765d4784b5
2,22,-0.611265,0.111135,-0.225794,0.661044,0.000000,9aeaad70b98f4bf0890734137f88c7ac
3,211,3.778570,0.205956,-1.089380,3.940330,0.139570,4261ccf933b64ce6ba83c623e9a5bf06
4,22,0.461909,0.330447,-0.168640,0.592448,0.000000,e79cf654192e4cab8b83f50554fededc


In [5]:
df[['particle_px', 'particle_py', 'particle_pz', 'particle_energy', 'particle_mass']].describe()

,particle_px,particle_py,particle_pz,particle_energy,particle_mass
count,24297352.000000,24297352.000000,24297352.000000,24297352.000000,24297352.000000
mean,-0.000589,0.000044,0.000086,7.441915,0.150297
std,15.863254,2.343419,2.301559,14.391967,0.237913
min,-453.928000,-346.806000,-347.040000,0.000021,0.000000
25%,-2.438860,-0.266196,-0.251602,1.082250,0.000000
50%,-0.002668,0.000050,0.000130,2.822150,0.139570
75%,2.433180,0.266330,0.252053,7.421680,0.139570
max,469.889000,253.947000,376.087000,470.319000,0.939570


In [6]:
def l2_dict(x, y, z): 
    return np.sqrt(np.square(x)+np.square(y)+np.square(z))

do_in_vec = np.vectorize(l2_dict, otypes=[np.float])
vec = do_in_vec(df.particle_px, df.particle_py, df.particle_pz)
df['particle_distance'] = vec

In [9]:
# vec = do_in_vec(df.particle_px, df.particle_py, 0)
# df['xy_dis'] = vec

vec = do_in_vec(0, df.particle_py, df.particle_pz)
df['yz_dis'] = vec

# vec = do_in_vec(df.particle_px, 0, df.particle_pz)
# df['zx_dis'] = vec

In [10]:
df['x_div_dist'] = df['particle_px'] / df['particle_distance']
# df['y_div_dist'] = df['particle_py'] / df['particle_distance']
# df['z_div_dist'] = df['particle_pz'] / df['particle_distance']
# df['xy_div_dist'] = df['xy_dis'] / df['particle_distance']
df['yz_div_dist'] = df['yz_dis'] / df['particle_distance']
# df['zx_div_dist'] = df['zx_dis'] / df['particle_distance']

In [11]:
df['energy_x'] = df['particle_energy'] * df['x_div_dist']
# df['energy_y'] = df['particle_energy'] * df['y_div_dist']
# df['energy_z'] = df['particle_energy'] * df['z_div_dist']
# df['energy_xy'] = df['particle_energy'] * df['xy_div_dist']
# df['energy_yz'] = df['particle_energy'] * df['yz_div_dist']
# df['energy_zx'] = df['particle_energy'] * df['zx_div_dist']

In [12]:
df['mass_x'] = df['particle_mass'] * df['x_div_dist']
# df['mass_y'] = df['particle_mass'] * df['y_div_dist']
# df['mass_z'] = df['particle_mass'] * df['z_div_dist']
# df['mass_xy'] = df['particle_mass'] * df['xy_div_dist']
df['mass_yz'] = df['particle_mass'] * df['yz_div_dist']
# df['mass_zx'] = df['particle_mass'] * df['zx_div_dist']

In [13]:
# def angle(x, y):
#     return np.degrees(np.math.atan(x / y)) if y != 0 else 0

# do_in_vec = np.vectorize(angle, otypes=[np.float])

# vec = do_in_vec(df.particle_px, df.particle_py)
# df['angle_xy'] = vec
# vec = do_in_vec(df.particle_py, df.particle_px)
# df['angle_yx'] = vec
# vec = do_in_vec(df.particle_py, df.particle_pz)
# df['angle_yz'] = vec
# vec = do_in_vec(df.particle_pz, df.particle_py)
# df['angle_zy'] = vec
# vec = do_in_vec(df.particle_pz, df.particle_px)
# df['angle_zx'] = vec
# vec = do_in_vec(df.particle_px, df.particle_pz)
# df['angle_xz'] = vec

In [14]:
def calculate_speed(e, m):
    return np.sqrt(2*e/m) if m > 0 else 0

do_in_vec = np.vectorize(calculate_speed, otypes=[np.float])
vec = do_in_vec(df.particle_energy, df.particle_mass)
df['particle_speed'] = vec

In [15]:
df['speed_x'] = df['particle_speed'] * df['x_div_dist']
# df['speed_y'] = df['particle_speed'] * df['y_div_dist']
# df['speed_z'] = df['particle_speed'] * df['z_div_dist']
# df['speed_xy'] = df['particle_speed'] * df['xy_div_dist']
# df['speed_yz'] = df['particle_speed'] * df['yz_div_dist']
# df['speed_zx'] = df['particle_speed'] * df['zx_div_dist']

In [16]:
def calculate_travel_time(d, v):
    return np.abs(d) / v if v > 0 else 0

# do_in_vec = np.vectorize(calculate_travel_time, otypes=[np.float])

# vec = do_in_vec(df.particle_distance, df.particle_speed)
# df['time_dis'] = vec

vec = do_in_vec(df.particle_px, df.speed_x)
df['time_x'] = vec

# vec = do_in_vec(df.particle_py, df.speed_y)
# df['time_y'] = vec

# vec = do_in_vec(df.particle_pz, df.speed_z)
# df['time_z'] = vec

# vec = do_in_vec(df.xy_dis, df.speed_xy)
# df['time_xy'] = vec

# vec = do_in_vec(df.yz_dis, df.speed_yz)
# df['time_yz'] = vec

# vec = do_in_vec(df.zx_dis, df.speed_zx)
# df['time_zx'] = vec

In [17]:
# df['particle_mv'] = df['particle_mass'] * df['particle_speed']
# df['mv_x'] = df['particle_mv'] * df['x_div_dist']
# df['mv_y'] = df['particle_mv'] * df['y_div_dist']
# df['mv_z'] = df['particle_mv'] * df['z_div_dist']
# df['mv_xy'] = df['particle_mv'] * df['xy_div_dist']
# df['mv_yz'] = df['particle_mv'] * df['yz_div_dist']
# df['mv_zx'] = df['particle_mv'] * df['zx_div_dist']

In [18]:
def brute_force(df):
    
#     df['jet_id_mass_max'] = df.groupby('jet_id')['particle_mass'].transform('max')
#     df['jet_id_mass_mean'] = df.groupby('jet_id')['particle_mass'].transform('mean')
#     df['jet_id_mass_min'] = df.groupby('jet_id')['particle_mass'].transform('min')
#     df['jet_id_mass_std'] = df.groupby('jet_id')['particle_mass'].transform('std')
    
#     df['jet_id_energy_max'] = df.groupby('jet_id')['particle_energy'].transform('max')
#     df['jet_id_energy_mean'] = df.groupby('jet_id')['particle_energy'].transform('mean')
#     df['jet_id_energy_min'] = df.groupby('jet_id')['particle_energy'].transform('min')
#     df['jet_id_energy_std'] = df.groupby('jet_id')['particle_energy'].transform('std')
    
#     df['jet_id_mass_x_max'] = df.groupby('jet_id')['mass_x'].transform('max')
    df['jet_id_mass_x_mean'] = df.groupby('jet_id')['mass_x'].transform('mean')
    df['jet_id_mass_x_min'] = df.groupby('jet_id')['mass_x'].transform('min')
    df['jet_id_mass_x_std'] = df.groupby('jet_id')['mass_x'].transform('std')
    
#     df['jet_id_mass_y_max'] = df.groupby('jet_id')['mass_y'].transform('max')
#     df['jet_id_mass_y_mean'] = df.groupby('jet_id')['mass_y'].transform('mean')
#     df['jet_id_mass_y_min'] = df.groupby('jet_id')['mass_y'].transform('min')
#     df['jet_id_mass_y_std'] = df.groupby('jet_id')['mass_y'].transform('std')
    
#     df['jet_id_mass_z_max'] = df.groupby('jet_id')['mass_z'].transform('max')
#     df['jet_id_mass_z_mean'] = df.groupby('jet_id')['mass_z'].transform('mean')
#     df['jet_id_mass_z_min'] = df.groupby('jet_id')['mass_z'].transform('min')
#     df['jet_id_mass_z_std'] = df.groupby('jet_id')['mass_z'].transform('std')
    
#     df['jet_id_mass_xy_max'] = df.groupby('jet_id')['mass_xy'].transform('max')
#     df['jet_id_mass_xy_mean'] = df.groupby('jet_id')['mass_xy'].transform('mean')
#     df['jet_id_mass_xy_min'] = df.groupby('jet_id')['mass_xy'].transform('min')
#     df['jet_id_mass_xy_std'] = df.groupby('jet_id')['mass_xy'].transform('std')
    
#     df['jet_id_mass_yz_max'] = df.groupby('jet_id')['mass_yz'].transform('max')
    df['jet_id_mass_yz_mean'] = df.groupby('jet_id')['mass_yz'].transform('mean')
#     df['jet_id_mass_yz_min'] = df.groupby('jet_id')['mass_yz'].transform('min')
    df['jet_id_mass_yz_std'] = df.groupby('jet_id')['mass_yz'].transform('std')
    
#     df['jet_id_mass_zx_max'] = df.groupby('jet_id')['mass_zx'].transform('max')
#     df['jet_id_mass_zx_mean'] = df.groupby('jet_id')['mass_zx'].transform('mean')
#     df['jet_id_mass_zx_min'] = df.groupby('jet_id')['mass_zx'].transform('min')
#     df['jet_id_mass_zx_std'] = df.groupby('jet_id')['mass_zx'].transform('std')
    
#     df['jet_id_energy_x_max'] = df.groupby('jet_id')['energy_x'].transform('max')
    df['jet_id_energy_x_mean'] = df.groupby('jet_id')['energy_x'].transform('mean')
    df['jet_id_energy_x_min'] = df.groupby('jet_id')['energy_x'].transform('min')
    df['jet_id_energy_x_std'] = df.groupby('jet_id')['energy_x'].transform('std')
    
#     df['jet_id_energy_y_max'] = df.groupby('jet_id')['energy_y'].transform('max')
#     df['jet_id_energy_y_mean'] = df.groupby('jet_id')['energy_y'].transform('mean')
#     df['jet_id_energy_y_min'] = df.groupby('jet_id')['energy_y'].transform('min')
#     df['jet_id_energy_y_std'] = df.groupby('jet_id')['energy_y'].transform('std')
    
#     df['jet_id_energy_z_max'] = df.groupby('jet_id')['energy_z'].transform('max')
#     df['jet_id_energy_z_mean'] = df.groupby('jet_id')['energy_z'].transform('mean')
#     df['jet_id_energy_z_min'] = df.groupby('jet_id')['energy_z'].transform('min')
#     df['jet_id_energy_z_std'] = df.groupby('jet_id')['energy_z'].transform('std')
    
#     df['jet_id_energy_xy_max'] = df.groupby('jet_id')['energy_xy'].transform('max')
#     df['jet_id_energy_xy_mean'] = df.groupby('jet_id')['energy_xy'].transform('mean')
#     df['jet_id_energy_xy_min'] = df.groupby('jet_id')['energy_xy'].transform('min')
#     df['jet_id_energy_xy_std'] = df.groupby('jet_id')['energy_xy'].transform('std')
    
#     df['jet_id_energy_yz_max'] = df.groupby('jet_id')['energy_yz'].transform('max')
#     df['jet_id_energy_yz_mean'] = df.groupby('jet_id')['energy_yz'].transform('mean')
#     df['jet_id_energy_yz_min'] = df.groupby('jet_id')['energy_yz'].transform('min')
#     df['jet_id_energy_yz_std'] = df.groupby('jet_id')['energy_yz'].transform('std')
    
#     df['jet_id_energy_zx_max'] = df.groupby('jet_id')['energy_zx'].transform('max')
#     df['jet_id_energy_zx_mean'] = df.groupby('jet_id')['energy_zx'].transform('mean')
#     df['jet_id_energy_zx_min'] = df.groupby('jet_id')['energy_zx'].transform('min')
#     df['jet_id_energy_zx_std'] = df.groupby('jet_id')['energy_zx'].transform('std')
    
#     df['jet_id_distance_max'] = df.groupby('jet_id')['particle_distance'].transform('max')
#     df['jet_id_distance_mean'] = df.groupby('jet_id')['particle_distance'].transform('mean')
#     df['jet_id_distance_min'] = df.groupby('jet_id')['particle_distance'].transform('min')
#     df['jet_id_distance_std'] = df.groupby('jet_id')['particle_distance'].transform('std')
    
#     df['jet_id_xy_dis_max'] = df.groupby('jet_id')['xy_dis'].transform('max')
#     df['jet_id_xy_dis_mean'] = df.groupby('jet_id')['xy_dis'].transform('mean')
#     df['jet_id_xy_dis_min'] = df.groupby('jet_id')['xy_dis'].transform('min')
#     df['jet_id_xy_dis_std'] = df.groupby('jet_id')['xy_dis'].transform('std')
    
#     df['jet_id_yz_dis_max'] = df.groupby('jet_id')['yz_dis'].transform('max')
#     df['jet_id_yz_dis_mean'] = df.groupby('jet_id')['yz_dis'].transform('mean')
#     df['jet_id_yz_dis_min'] = df.groupby('jet_id')['yz_dis'].transform('min')
#     df['jet_id_yz_dis_std'] = df.groupby('jet_id')['yz_dis'].transform('std')
    
#     df['jet_id_zx_dis_max'] = df.groupby('jet_id')['zx_dis'].transform('max')
#     df['jet_id_zx_dis_mean'] = df.groupby('jet_id')['zx_dis'].transform('mean')
#     df['jet_id_zx_dis_min'] = df.groupby('jet_id')['zx_dis'].transform('min')
#     df['jet_id_zx_dis_std'] = df.groupby('jet_id')['zx_dis'].transform('std')
    
#     df['jet_id_x_div_dist_max'] = df.groupby('jet_id')['x_div_dist'].transform('max')
#     df['jet_id_x_div_dist_mean'] = df.groupby('jet_id')['x_div_dist'].transform('mean')
#     df['jet_id_x_div_dist_min'] = df.groupby('jet_id')['x_div_dist'].transform('min')
#     df['jet_id_x_div_dist_std'] = df.groupby('jet_id')['x_div_dist'].transform('std')
    
#     df['jet_id_y_div_dist_max'] = df.groupby('jet_id')['y_div_dist'].transform('max')
#     df['jet_id_y_div_dist_mean'] = df.groupby('jet_id')['y_div_dist'].transform('mean')
#     df['jet_id_y_div_dist_min'] = df.groupby('jet_id')['y_div_dist'].transform('min')
#     df['jet_id_y_div_dist_std'] = df.groupby('jet_id')['y_div_dist'].transform('std')
    
#     df['jet_id_z_div_dist_max'] = df.groupby('jet_id')['z_div_dist'].transform('max')
#     df['jet_id_z_div_dist_mean'] = df.groupby('jet_id')['z_div_dist'].transform('mean')
#     df['jet_id_z_div_dist_min'] = df.groupby('jet_id')['z_div_dist'].transform('min')
#     df['jet_id_z_div_dist_std'] = df.groupby('jet_id')['z_div_dist'].transform('std')
    
#     df['jet_id_xy_div_dist_max'] = df.groupby('jet_id')['xy_div_dist'].transform('max')
#     df['jet_id_xy_div_dist_mean'] = df.groupby('jet_id')['xy_div_dist'].transform('mean')
#     df['jet_id_xy_div_dist_min'] = df.groupby('jet_id')['xy_div_dist'].transform('min')
#     df['jet_id_xy_div_dist_std'] = df.groupby('jet_id')['xy_div_dist'].transform('std')
    
#     df['jet_id_yz_div_dist_max'] = df.groupby('jet_id')['yz_div_dist'].transform('max')
#     df['jet_id_yz_div_dist_mean'] = df.groupby('jet_id')['yz_div_dist'].transform('mean')
#     df['jet_id_yz_div_dist_min'] = df.groupby('jet_id')['yz_div_dist'].transform('min')
#     df['jet_id_yz_div_dist_std'] = df.groupby('jet_id')['yz_div_dist'].transform('std')
    
#     df['jet_id_zx_div_dist_max'] = df.groupby('jet_id')['zx_div_dist'].transform('max')
#     df['jet_id_zx_div_dist_mean'] = df.groupby('jet_id')['zx_div_dist'].transform('mean')
#     df['jet_id_zx_div_dist_min'] = df.groupby('jet_id')['zx_div_dist'].transform('min')
#     df['jet_id_zx_div_dist_std'] = df.groupby('jet_id')['zx_div_dist'].transform('std')
    
#     df['jet_id_speed_max'] = df.groupby('jet_id')['particle_speed'].transform('max')
    df['jet_id_speed_mean'] = df.groupby('jet_id')['particle_speed'].transform('mean')
    df['jet_id_speed_min'] = df.groupby('jet_id')['particle_speed'].transform('min')
    df['jet_id_speed_std'] = df.groupby('jet_id')['particle_speed'].transform('std')
    
#     df['jet_id_speed_x_max'] = df.groupby('jet_id')['speed_x'].transform('max')
    df['jet_id_speed_x_mean'] = df.groupby('jet_id')['speed_x'].transform('mean')
    df['jet_id_speed_x_min'] = df.groupby('jet_id')['speed_x'].transform('min')
    df['jet_id_speed_x_std'] = df.groupby('jet_id')['speed_x'].transform('std')
    
#     df['jet_id_speed_y_max'] = df.groupby('jet_id')['speed_y'].transform('max')
#     df['jet_id_speed_y_mean'] = df.groupby('jet_id')['speed_y'].transform('mean')
#     df['jet_id_speed_y_min'] = df.groupby('jet_id')['speed_y'].transform('min')
#     df['jet_id_speed_y_std'] = df.groupby('jet_id')['speed_y'].transform('std')
    
#     df['jet_id_speed_z_max'] = df.groupby('jet_id')['speed_z'].transform('max')
#     df['jet_id_speed_z_mean'] = df.groupby('jet_id')['speed_z'].transform('mean')
#     df['jet_id_speed_z_min'] = df.groupby('jet_id')['speed_z'].transform('min')
#     df['jet_id_speed_z_std'] = df.groupby('jet_id')['speed_z'].transform('std')
    
#     df['jet_id_speed_xy_max'] = df.groupby('jet_id')['speed_xy'].transform('max')
#     df['jet_id_speed_xy_mean'] = df.groupby('jet_id')['speed_xy'].transform('mean')
#     df['jet_id_speed_xy_min'] = df.groupby('jet_id')['speed_xy'].transform('min')
#     df['jet_id_speed_xy_std'] = df.groupby('jet_id')['speed_xy'].transform('std')
    
#     df['jet_id_speed_yz_max'] = df.groupby('jet_id')['speed_yz'].transform('max')
#     df['jet_id_speed_yz_mean'] = df.groupby('jet_id')['speed_yz'].transform('mean')
#     df['jet_id_speed_yz_min'] = df.groupby('jet_id')['speed_yz'].transform('min')
#     df['jet_id_speed_yz_std'] = df.groupby('jet_id')['speed_yz'].transform('std')
    
#     df['jet_id_speed_zx_max'] = df.groupby('jet_id')['speed_zx'].transform('max')
#     df['jet_id_speed_zx_mean'] = df.groupby('jet_id')['speed_zx'].transform('mean')
#     df['jet_id_speed_zx_min'] = df.groupby('jet_id')['speed_zx'].transform('min')
#     df['jet_id_speed_zx_std'] = df.groupby('jet_id')['speed_zx'].transform('std')    
    
#     df['jet_id_px_max'] = df.groupby('jet_id')['particle_px'].transform('max')
    df['jet_id_px_mean'] = df.groupby('jet_id')['particle_px'].transform('mean')
    df['jet_id_px_min'] = df.groupby('jet_id')['particle_px'].transform('min')
    df['jet_id_px_std'] = df.groupby('jet_id')['particle_px'].transform('std')
    
#     df['jet_id_py_max'] = df.groupby('jet_id')['particle_py'].transform('max')
#     df['jet_id_py_mean'] = df.groupby('jet_id')['particle_py'].transform('mean')
#     df['jet_id_py_min'] = df.groupby('jet_id')['particle_py'].transform('min')
#     df['jet_id_py_std'] = df.groupby('jet_id')['particle_py'].transform('std')
    
#     df['jet_id_pz_max'] = df.groupby('jet_id')['particle_pz'].transform('max')
#     df['jet_id_pz_mean'] = df.groupby('jet_id')['particle_pz'].transform('mean')
#     df['jet_id_pz_min'] = df.groupby('jet_id')['particle_pz'].transform('min')
#     df['jet_id_pz_std'] = df.groupby('jet_id')['particle_pz'].transform('std')
    
#     df['jet_id_angle_xy_max'] = df.groupby('jet_id')['angle_xy'].transform('max')
#     df['jet_id_angle_xy_mean'] = df.groupby('jet_id')['angle_xy'].transform('mean')
#     df['jet_id_angle_xy_min'] = df.groupby('jet_id')['angle_xy'].transform('min')
#     df['jet_id_angle_xy_std'] = df.groupby('jet_id')['angle_xy'].transform('std')
    
#     df['jet_id_angle_xz_max'] = df.groupby('jet_id')['angle_xz'].transform('max')
#     df['jet_id_angle_xz_mean'] = df.groupby('jet_id')['angle_xz'].transform('mean')
#     df['jet_id_angle_xz_min'] = df.groupby('jet_id')['angle_xz'].transform('min')
#     df['jet_id_angle_xz_std'] = df.groupby('jet_id')['angle_xz'].transform('std')
    
#     df['jet_id_angle_yx_max'] = df.groupby('jet_id')['angle_yx'].transform('max')
#     df['jet_id_angle_yx_mean'] = df.groupby('jet_id')['angle_yx'].transform('mean')
#     df['jet_id_angle_yx_min'] = df.groupby('jet_id')['angle_yx'].transform('min')
#     df['jet_id_angle_yx_std'] = df.groupby('jet_id')['angle_yx'].transform('std')
    
#     df['jet_id_angle_yz_max'] = df.groupby('jet_id')['angle_yz'].transform('max')
#     df['jet_id_angle_yz_mean'] = df.groupby('jet_id')['angle_yz'].transform('mean')
#     df['jet_id_angle_yz_min'] = df.groupby('jet_id')['angle_yz'].transform('min')
#     df['jet_id_angle_yz_std'] = df.groupby('jet_id')['angle_yz'].transform('std')
    
#     df['jet_id_angle_zy_max'] = df.groupby('jet_id')['angle_zy'].transform('max')
#     df['jet_id_angle_zy_mean'] = df.groupby('jet_id')['angle_zy'].transform('mean')
#     df['jet_id_angle_zy_min'] = df.groupby('jet_id')['angle_zy'].transform('min')
#     df['jet_id_angle_zy_std'] = df.groupby('jet_id')['angle_zy'].transform('std')
    
#     df['jet_id_angle_zx_max'] = df.groupby('jet_id')['angle_zx'].transform('max')
#     df['jet_id_angle_zx_mean'] = df.groupby('jet_id')['angle_zx'].transform('mean')
#     df['jet_id_angle_zx_min'] = df.groupby('jet_id')['angle_zx'].transform('min')
#     df['jet_id_angle_zx_std'] = df.groupby('jet_id')['angle_zx'].transform('std')
    
#     df['jet_id_time_dis_max'] = df.groupby('jet_id')['time_dis'].transform('max')
#     df['jet_id_time_dis_mean'] = df.groupby('jet_id')['time_dis'].transform('mean')
#     df['jet_id_time_dis_min'] = df.groupby('jet_id')['time_dis'].transform('min')
#     df['jet_id_time_dis_std'] = df.groupby('jet_id')['time_dis'].transform('std')
    
#     df['jet_id_time_x_max'] = df.groupby('jet_id')['time_x'].transform('max')
    df['jet_id_time_x_mean'] = df.groupby('jet_id')['time_x'].transform('mean')
    df['jet_id_time_x_min'] = df.groupby('jet_id')['time_x'].transform('min')
    df['jet_id_time_x_std'] = df.groupby('jet_id')['time_x'].transform('std')
    
#     df['jet_id_time_y_max'] = df.groupby('jet_id')['time_y'].transform('max')
#     df['jet_id_time_y_mean'] = df.groupby('jet_id')['time_y'].transform('mean')
#     df['jet_id_time_y_min'] = df.groupby('jet_id')['time_y'].transform('min')
#     df['jet_id_time_y_std'] = df.groupby('jet_id')['time_y'].transform('std')
    
#     df['jet_id_time_z_max'] = df.groupby('jet_id')['time_z'].transform('max')
#     df['jet_id_time_z_mean'] = df.groupby('jet_id')['time_z'].transform('mean')
#     df['jet_id_time_z_min'] = df.groupby('jet_id')['time_z'].transform('min')
#     df['jet_id_time_z_std'] = df.groupby('jet_id')['time_z'].transform('std')
    
#     df['jet_id_time_xy_max'] = df.groupby('jet_id')['time_xy'].transform('max')
#     df['jet_id_time_xy_mean'] = df.groupby('jet_id')['time_xy'].transform('mean')
#     df['jet_id_time_xy_min'] = df.groupby('jet_id')['time_xy'].transform('min')
#     df['jet_id_time_xy_std'] = df.groupby('jet_id')['time_xy'].transform('std')
    
#     df['jet_id_time_yz_max'] = df.groupby('jet_id')['time_yz'].transform('max')
#     df['jet_id_time_yz_mean'] = df.groupby('jet_id')['time_yz'].transform('mean')
#     df['jet_id_time_yz_min'] = df.groupby('jet_id')['time_yz'].transform('min')
#     df['jet_id_time_yz_std'] = df.groupby('jet_id')['time_yz'].transform('std')
    
#     df['jet_id_time_zx_max'] = df.groupby('jet_id')['time_zx'].transform('max')
#     df['jet_id_time_zx_mean'] = df.groupby('jet_id')['time_zx'].transform('mean')
#     df['jet_id_time_zx_min'] = df.groupby('jet_id')['time_zx'].transform('min')
#     df['jet_id_time_zx_std'] = df.groupby('jet_id')['time_zx'].transform('std')
    
#     df['jet_id_mv_max'] = df.groupby('jet_id')['jet_mv'].transform('max')
#     df['jet_id_mv_mean'] = df.groupby('jet_id')['jet_mv'].transform('mean')
#     df['jet_id_mv_min'] = df.groupby('jet_id')['jet_mv'].transform('min')
#     df['jet_id_mv_std'] = df.groupby('jet_id')['jet_mv'].transform('std')
    
#     df['jet_id_mv_x_max'] = df.groupby('jet_id')['mv_x'].transform('max')
#     df['jet_id_mv_x_mean'] = df.groupby('jet_id')['mv_x'].transform('mean')
#     df['jet_id_mv_x_min'] = df.groupby('jet_id')['mv_x'].transform('min')
#     df['jet_id_mv_x_std'] = df.groupby('jet_id')['mv_x'].transform('std')
    
#     df['jet_id_mv_y_max'] = df.groupby('jet_id')['mv_y'].transform('max')
#     df['jet_id_mv_y_mean'] = df.groupby('jet_id')['mv_y'].transform('mean')
#     df['jet_id_mv_y_min'] = df.groupby('jet_id')['mv_y'].transform('min')
#     df['jet_id_mv_y_std'] = df.groupby('jet_id')['mv_y'].transform('std')
    
#     df['jet_id_mv_z_max'] = df.groupby('jet_id')['mv_z'].transform('max')
#     df['jet_id_mv_z_mean'] = df.groupby('jet_id')['mv_z'].transform('mean')
#     df['jet_id_mv_z_min'] = df.groupby('jet_id')['mv_z'].transform('min')
#     df['jet_id_mv_z_std'] = df.groupby('jet_id')['mv_z'].transform('std')
    
#     df['jet_id_mv_xy_max'] = df.groupby('jet_id')['mv_xy'].transform('max')
#     df['jet_id_mv_xy_mean'] = df.groupby('jet_id')['mv_xy'].transform('mean')
#     df['jet_id_mv_xy_min'] = df.groupby('jet_id')['mv_xy'].transform('min')
#     df['jet_id_mv_xy_std'] = df.groupby('jet_id')['mv_xy'].transform('std')
    
#     df['jet_id_mv_yz_max'] = df.groupby('jet_id')['mv_yz'].transform('max')
#     df['jet_id_mv_yz_mean'] = df.groupby('jet_id')['mv_yz'].transform('mean')
#     df['jet_id_mv_yz_min'] = df.groupby('jet_id')['mv_yz'].transform('min')
#     df['jet_id_mv_yz_std'] = df.groupby('jet_id')['mv_yz'].transform('std')
    
#     df['jet_id_mv_zx_max'] = df.groupby('jet_id')['mv_zx'].transform('max')
#     df['jet_id_mv_zx_mean'] = df.groupby('jet_id')['mv_zx'].transform('mean')
#     df['jet_id_mv_zx_min'] = df.groupby('jet_id')['mv_zx'].transform('min')
#     df['jet_id_mv_zx_std'] = df.groupby('jet_id')['mv_zx'].transform('std')
     
    return df

In [19]:
df = brute_force(df)

In [20]:
df['jet_id_energy_sum'] = df.groupby('jet_id')['particle_energy'].transform('sum')
df['jet_id_mass_sum'] = df.groupby('jet_id')['particle_mass'].transform('sum')

In [21]:
do_in_vec = np.vectorize(calculate_speed, otypes=[np.float])
vec = do_in_vec(df.jet_id_energy_sum, df.jet_id_mass_sum)
df['jet_id_speed'] = vec

In [22]:
to_drop_cols = [
    'particle_distance', 'yz_dis', 'x_div_dist', 'yz_div_dist', 'speed_x', 'time_x', 
]

df = df.drop(to_drop_cols, axis=1)

In [23]:
df.shape

(24297352, 34)

In [25]:
df['particle_category'] = df['particle_category'].astype('int16')

for col in tqdm([i for i in df.columns.tolist() if i not in ['particle_category', 'jet_id']]):
    df[col] = df[col].astype('float16')

100%|██████████| 32/32 [00:17<00:00,  5.43it/s]


In [26]:
gc.collect()

44

In [30]:
df = df.reindex(columns=[i for i in df.columns.tolist() if i != 'jet_id'] + ['jet_id'])

In [32]:
df.head()

,particle_category,particle_px,particle_py,particle_pz,particle_energy,particle_mass,energy_x,mass_x,mass_yz,particle_speed,jet_id_mass_x_mean,jet_id_mass_x_min,jet_id_mass_x_std,jet_id_mass_yz_mean,jet_id_mass_yz_std,jet_id_energy_x_mean,jet_id_energy_x_min,jet_id_energy_x_std,jet_id_speed_mean,jet_id_speed_min,jet_id_speed_std,jet_id_speed_x_mean,jet_id_speed_x_min,jet_id_speed_x_std,jet_id_px_mean,jet_id_px_min,jet_id_px_std,jet_id_time_x_mean,jet_id_time_x_min,jet_id_time_x_std,jet_id_energy_sum,jet_id_mass_sum,jet_id_speed,jet_id
0,-211,-1.299805,0.368652,-0.057007,1.359375,0.139526,-1.306641,-0.134155,0.038513,4.414062,-0.203979,-0.936523,0.274902,0.034485,0.043243,-3.859375,-25.015625,5.835938,3.121094,0.000000,3.419922,-3.066406,-11.320312,3.398438,-3.841797,-25.000000,5.835938,0.000000,0.000000,0.000000,105.437500,5.613281,6.128906,01291194f90c44c7bd79d9dbd50abd93
1,-2212,1.305664,0.418213,-0.426025,1.714844,0.938477,1.559570,0.853027,0.390381,1.912109,0.142334,0.000000,0.235962,0.028809,0.061584,9.484375,0.149536,16.078125,5.250000,0.000000,7.230469,5.164062,0.000000,7.136719,9.476562,0.149536,16.078125,0.731445,0.000000,0.857422,481.000000,7.304688,11.476562,7df78235aa464796bbb386765d4784b5
2,22,-0.611328,0.111145,-0.225830,0.661133,0.000000,-0.611328,-0.000000,0.000000,0.000000,-0.081299,-0.480469,0.100586,0.018036,0.024231,-7.589844,-63.875000,13.687500,4.824219,0.000000,6.238281,-4.699219,-25.625000,6.078125,-7.585938,-63.875000,13.687500,0.000000,0.000000,0.000000,241.500000,2.587891,13.664062,9aeaad70b98f4bf0890734137f88c7ac
3,211,3.779297,0.205933,-1.089844,3.939453,0.139526,3.781250,0.133911,0.039307,7.515625,0.153809,0.000000,0.233276,0.031281,0.055542,6.671875,0.223511,8.890625,4.742188,0.000000,5.300781,4.652344,0.000000,5.230469,6.664062,0.223511,8.890625,0.749023,0.000000,0.783203,358.750000,8.359375,9.265625,4261ccf933b64ce6ba83c623e9a5bf06
4,22,0.461914,0.330566,-0.168579,0.592285,0.000000,0.461914,0.000000,0.000000,0.000000,0.065491,0.000000,0.113464,0.049438,0.086182,4.511719,0.195923,5.390625,7.128906,0.000000,15.914062,5.675781,0.000000,12.718750,4.507812,0.195923,5.390625,0.467285,0.000000,0.619629,199.875000,2.876953,11.789062,e79cf654192e4cab8b83f50554fededc


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24297352 entries, 0 to 24297351
Data columns (total 34 columns):
particle_category       int16
particle_px             float16
particle_py             float16
particle_pz             float16
particle_energy         float16
particle_mass           float16
energy_x                float16
mass_x                  float16
mass_yz                 float16
particle_speed          float16
jet_id_mass_x_mean      float16
jet_id_mass_x_min       float16
jet_id_mass_x_std       float16
jet_id_mass_yz_mean     float16
jet_id_mass_yz_std      float16
jet_id_energy_x_mean    float16
jet_id_energy_x_min     float16
jet_id_energy_x_std     float16
jet_id_speed_mean       float16
jet_id_speed_min        float16
jet_id_speed_std        float16
jet_id_speed_x_mean     float16
jet_id_speed_x_min      float16
jet_id_speed_x_std      float16
jet_id_px_mean          float16
jet_id_px_min           float16
jet_id_px_std           float16
jet_id_time_x_mean     

In [33]:
df.to_pickle('train_particle_sm.pickle')

In [34]:
for i, col in enumerate(df.columns.tolist()):
    print(i, col)

0 particle_category
1 particle_px
2 particle_py
3 particle_pz
4 particle_energy
5 particle_mass
6 energy_x
7 mass_x
8 mass_yz
9 particle_speed
10 jet_id_mass_x_mean
11 jet_id_mass_x_min
12 jet_id_mass_x_std
13 jet_id_mass_yz_mean
14 jet_id_mass_yz_std
15 jet_id_energy_x_mean
16 jet_id_energy_x_min
17 jet_id_energy_x_std
18 jet_id_speed_mean
19 jet_id_speed_min
20 jet_id_speed_std
21 jet_id_speed_x_mean
22 jet_id_speed_x_min
23 jet_id_speed_x_std
24 jet_id_px_mean
25 jet_id_px_min
26 jet_id_px_std
27 jet_id_time_x_mean
28 jet_id_time_x_min
29 jet_id_time_x_std
30 jet_id_energy_sum
31 jet_id_mass_sum
32 jet_id_speed
33 jet_id
